In [1]:
from variable.variable import Variable
from activation_functions.relu import ReLUActivation
relu_fn = ReLUActivation()

In [2]:
x = Variable(3)
y = Variable(4)

In [3]:
z = x + y 

In [4]:
from models.simple_model import SimpleModel
input_size = 3
hidden_size = 10
output_size = 1


model = SimpleModel(input_size, hidden_size, output_size)

In [5]:
from optimizer.sgd import SGDOptimizer
from loss_functions.mse import MSELoss
from variable.variable import Variable
import random

loss_fn = MSELoss()

optimizer = SGDOptimizer(model.parameters(), learning_rate=0.001)

losses = []
#let's feed random input and make the model always output 0

for i in range(100):
    x = [[Variable(random.randint(0, 10)) for i in range(input_size)]]
    y = [[Variable(0)]]
    output = model(x)
    loss = loss_fn(output, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)
    losses.append(loss.loss)


Loss: 2211.638998782471
Loss: 468.93854419957427
Loss: 103.66440959837315
Loss: 1.5556203300733875
Loss: 4.492550039955883
Loss: 39.44688731610975
Loss: 114.21772793119447
Loss: 6.599731791453235
Loss: 1.515647816596179
Loss: 4.955551427494127
Loss: 3.350454530136938
Loss: 3.6201665978602624
Loss: 0.22233557550401586
Loss: 0.010012138350658941
Loss: 0.9562199900842665
Loss: 0.4568474432107186
Loss: 1.4637975295149568
Loss: 0.6914694758046793
Loss: 2.9663406984731338
Loss: 1.5400060458652545
Loss: 0.07148817012350978
Loss: 0.4322274237603232
Loss: 0.056662003397526396
Loss: 0.9657914417574902
Loss: 0.0008075741992811833
Loss: 0.09758294041460006
Loss: 0.012033303301140257
Loss: 3.4344235875764735
Loss: 0.663284131474585
Loss: 1.2072487331856758
Loss: 0.05296311766670792
Loss: 0.8798852926284862
Loss: 0.02474343345879132
Loss: 0.38055066746680016
Loss: 0.04988329970728549
Loss: 0.5173580484330844
Loss: 0.4134686021390968
Loss: 0.016951171255568134
Loss: 0.30604395819855484
Loss: 2.317784

In [6]:
#plot the losses
import matplotlib.pyplot as plt
plt.plot(losses)
plt.ylabel('loss')
plt.xlabel('iteration')
plt.show()

